In [2]:
from cyclic import cadd, cmul


def add(a: int, b: int):
    return (a + b) % M


def mul(a: int, b: int):
    return (a * b) % M


# best period: M (if P is not 0)
def P_add_a(a: int):
    return add(P, a)


# best period: M (if P is not 0)
def c_P_add_a(a: int):
    return cadd(P, a, S)


# best period: M (if P is mutual prime to M and is not 0)
def P_mul_a(a: int):
    return mul(P, a)


# best period: M^2 (if P is not 0 - ya just not a 0 wow!)
def c_P_mul_a(a: int):
    return cmul(P, a, S)


# NOTE: it gives very symmetric image, but when selecting each M'th row, the image is very random
# best so far found period: M/2 ~ M
def a_mul_a(a: int):
    return mul(a, a)


# best so far found period: M*(M-1) ~ M^2
def c_a_mul_a(a: int):
    return cmul(a, a, S)


# best so far found period: M
def a_to_P(a: int):
    # returns a^P
    res = 1
    for bit in bin(P)[2:]:
        res = mul(res, res)
        if bit == "1":
            res = mul(res, a)
    return res


# best so far found period: M
def c_a_to_P(a: int):
    # returns a^P
    res = 1
    for bit in bin(P)[2:]:
        res = cmul(res, res)
        if bit == "1":
            res = cmul(res, a)
    return res


# best so far found period: 0.25M ~ M
def P_to_a(a: int):
    # returns P^a
    res = 1
    for bit in bin(a)[2:]:
        res = mul(res, res)
        if bit == "1":
            res = mul(res, P)
    return res


# TODO: find ways to solve such discrete log
# TODO: find a way to calculate the period for this function without full enumeration (recall how it's done in a regular case with euler's theorem)
# best so far found period: 0.9M ~ M
def c_P_to_a(a: int):
    # returns cyclic P^a
    res = 1
    for bit in bin(a)[2:]:
        res = cmul(res, res, S)
        if bit == "1":
            res = cmul(res, P, S)
    return res


# best so far found period: M, but on S > 6 didnt find yet
def a_to_a(a: int):
    # returns a^a
    res = 1
    for bit in bin(a)[2:]:
        res = mul(res, res)
        if bit == "1":
            res = mul(res, a)
    return res


# it definitely has no period it's emulating something that is not addition by modulus
# TODO: find ways to solve such discrete sqrt (both quantum and classical)
def c_a_to_a(a: int):
    # returns cyclic a^a
    res = 1
    for bit in bin(a)[2:]:
        res = cmul(res, res, S)
        if bit == "1":
            res = cmul(res, a, S)
    return res

In [5]:
# just visualize
S = 5
M = pow(2, 5)
a = ["-"] * M
P = 5
for c in range(5):
    v = P_to_a(c)
    a = ["-"] * M
    a[v] = "x"
    print(" ".join(a), "<" if c % M == 0 else "", v)

- x - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - < 1
- - - - - x - - - - - - - - - - - - - - - - - - - - - - - - - -  5
- - - - - - - - - - - - - - - - - - - - - - - - - x - - - - - -  25
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - x - -  29
- - - - - - - - - - - - - - - - - x - - - - - - - - - - - - - -  17


In [3]:
# Check rules
# Checking the real period: f(p) = 0, f(x + p) = 0

S = 6
M = 2**S
x = 2
y = 7

Ex = c_a_mul_a(x)
Ey = c_a_mul_a(y)

res = (
    x * y
)  # TODO: find f(x, y) for c_a_mul_a this f(x, y) = x*y % p is definitely not a case

Eres = c_a_mul_a(res)

print(Eres - cmul(Ex, Ey))

-3


In [4]:
# check period
# NOTE: when applying to c_* functions you can see a fake period

S = 6
M = 2**S
print("M:", M)
for p in range(M, M * M):
    for x in range(2, p):
        try:
            x_1 = pow(x, -1, p)
        except:
            continue
        r = mul(a_mul_a(x), a_mul_a(x_1))
        if r != 1:
            break
    else:
        print(p)

M: 64
64
96
128
160
192
224
256
288
320
352
384
416
448
480
512
544
576
608
640
672
704
736
768
800
832
864
896
928
960
992
1024
1056
1088
1120
1152
1184
1216
1248
1280
1312
1344
1376
1408
1440
1472
1504
1536
1568
1600
1632
1664
1696
1728
1760
1792
1824
1856
1888
1920
1952
1984
2016
2048
2080
2112
2144
2176
2208
2240
2272
2304
2336
2368
2400
2432
2464
2496
2528
2560
2592
2624
2656
2688
2720
2752
2784
2816
2848
2880
2912
2944
2976
3008
3040
3072
3104
3136
3168
3200
3232
3264
3296
3328
3360
3392
3424
3456
3488
3520
3552
3584
3616
3648
3680
3712
3744
3776
3808
3840
3872
3904
3936
3968
4000
4032
4064


In [10]:
# search for the best period when P is also iterable
# NOTE: when applying to c_* functions finds a fake period
from typing import List


best = (-1, -1, -1)
shift = 3
for S in range(5, 6):
    M = 2**S
    for P in range(1, M):
        print(f"Searching period for (S, P) = ({S}, {P})...")

        values: List[int] = []
        found = False
        for c in range(
            shift, M**M + 4 + shift
        ):  # assuming no greater than this value period can be found
            v = a_to_a(c)
            values.append(v)
            p = len(values) // 2
            if len(values) % 2 == 0 and values[:p] == values[p:]:
                print(p, "that is", f"{p/M}*M,", f"M = {M}")
                if p > best[2]:
                    best = S, P, p
                found = True
                break

        if not found:
            print("No period found")

if best[0] != -1:
    S, P, p = best
    M = 2**S
    print(
        f"Best found: (S, P) = ({S}, {P}) with period = {p}",
        "that is",
        f"{p/M}*M,",
        f"M = {M}",
    )

Searching period for (S, P) = (5, 1)...
32 that is 1.0*M, M = 32
Searching period for (S, P) = (5, 2)...
32 that is 1.0*M, M = 32
Searching period for (S, P) = (5, 3)...
32 that is 1.0*M, M = 32
Searching period for (S, P) = (5, 4)...
32 that is 1.0*M, M = 32
Searching period for (S, P) = (5, 5)...
32 that is 1.0*M, M = 32
Searching period for (S, P) = (5, 6)...
32 that is 1.0*M, M = 32
Searching period for (S, P) = (5, 7)...
32 that is 1.0*M, M = 32
Searching period for (S, P) = (5, 8)...
32 that is 1.0*M, M = 32
Searching period for (S, P) = (5, 9)...
32 that is 1.0*M, M = 32
Searching period for (S, P) = (5, 10)...
32 that is 1.0*M, M = 32
Searching period for (S, P) = (5, 11)...
32 that is 1.0*M, M = 32
Searching period for (S, P) = (5, 12)...
32 that is 1.0*M, M = 32
Searching period for (S, P) = (5, 13)...
32 that is 1.0*M, M = 32
Searching period for (S, P) = (5, 14)...
32 that is 1.0*M, M = 32
Searching period for (S, P) = (5, 15)...
32 that is 1.0*M, M = 32
Searching period fo

In [13]:
# search for the best period when P is not iterable
# NOTE: when applying to c_* functions finds a fake period

P = 3
best = (-1, -1)
for S in range(4, 5):
    M = 2**S
    print(f"Searching period for S = {S}...")

    values: List[int] = []
    found = False
    for c in range(M * M * M * 2):  # assuming no greater than M^3 period can be found
        v = c_a_mul_a(c)
        values.append(v)
        if (
            len(values) % 2 == 0
            and values[: len(values) // 2] == values[len(values) // 2 :]
        ):
            p = len(values) // 2
            print(
                p,
                "that is",
                f"{p/M}*M,",
                f"M = {M}",
            )
            if p > best[1]:
                best = S, p
            found = True
            break

    if not found:
        print("No period")

if best[0] != -1:
    S, p = best
    M = 2**S
    print(
        f"Best found: S = {S} with period = {p}",
        "that is",
        f"{p/M}*M,",
        f"M = {M}",
    )


Searching period for S = 4...
240 that is 15.0*M, M = 16
Best found: S = 4 with period = 240 that is 15.0*M, M = 16
